In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensornetwork as tn
import numpy as np
from tensorflow.keras.datasets import mnist
import time

tn.set_default_backend("numpy")

In [2]:
# The following code construct a MPS like this

#                        label_size
#                            |
    # bond_dim               |
# A1 --------- A2 ---------- L --------- A3 --------- An
# |            |                         |            |
# |            |                         |            |
#           feature      
def construct_MPS(bond_dim, feature, rank, label_size):
    label_site = rank // 2
    mps = tn.InfiniteMPS.random(
        d = [feature for _ in range(label_site)] + [label_size] + [feature for _ in range(rank - label_site)],
        D = [bond_dim for _ in range(rank+2)],
        dtype = np.float32
    )

    # left_region = np.stack(mps.tensors[:label_site], axis=0)
    # output_site = mps.tensors[label_site]
    # right_region = np.stack(mps.tensors[label_site+1:], axis=0)
    
    return mps
    # return nodes, connected_bonds

In [3]:
class MPSModle(tf.keras.Model):
    def __init__(self, bond_dim, feature, rank, label_size):
        super(MPSModle, self).__init__()
        mps = construct_MPS(bond_dim, feature, rank, label_size)
        label_site = rank // 2
        left_input_tensors = mps.tensors[:label_site]
        output_tensor = mps.tensors[label_site]
        right_input_tensors = mps.tensors[label_site+1:]

        left_input_region = InputRegion(left_input_tensors)
        outputM = OutputModule(output_tensor)
        right_input_region = InputRegion(right_input_tensors)

        self.left_input_region = left_input_region
        self.outputM = outputM
        self.right_input_region = right_input_region

    # def __call__(self, x):
    #     return self.a_variable * x + self.non_trainable_variable

class InputModule(tf.keras.Model):
    def __init__(self, tensor, name=None):
        super().__init__(name=name)
        # tensor [l, feature, r]
        self.input_tensor = tf.Variable(tensor, trainable=True)

    def __call__(self, input):
        # input [feature]
        core = tn.Node(self.input_tensor.numpy())
        input_node = tn.Node(input)
        core.edges[1] ^ input_node[0]
        result = core @ input_node
        return result

class OutputModule(tf.keras.Model):
    def __init__(self, tensor, name=None):
        super().__init__(name=name)
        # tensor [l, label_size, r]
        self.output_tensor = tf.Variable(tensor, trainable=True)

    def __call__(self):        
        return tn.Node(self.output_tensor.numpy())

class InputRegion(tf.keras.Model):
    def __init__(self, tensors, name=None):
        super().__init__(name=name)
        # tensors [[l, feature, r]]
        input_Modules = [InputModule(tensor) for tensor in tensors]

        self.tensors = tensors
        self.input_Modules = input_Modules
    
    def __call__(self, input):
        # input [rank, feature]
        assert input.shape[0] == len(self.input_Modules)
        # nodes [[l,r]]
        nodes = [module(tensor) for module, tensor in zip(self.input_Modules, input)]
        # rel [l,r]
        for i in range(len(nodes) - 1):
            nodes[i].edges[1] ^ nodes[i+1].edges[0]
            rel = nodes[i] @ nodes[i+1]
        return rel

In [4]:
print("######################################")
print("FOR inputModule Test")
input_tensor = np.random.random_sample([3,2,3])
inputM = InputModule(input_tensor)
print(inputM.variables)
print(inputM(np.ones(2)).tensor.shape)

print("\n######################################")
print("FOR outputModule Test")
output_tensor = np.random.random_sample([3,10,3])
outputM = OutputModule(output_tensor)
print(outputM().tensor.shape[1])

print("\n######################################")
print("FOR inputRegion Test")
tensors = [np.random.random_sample([3,2,3]) for _ in range(10)]
inputRegion = InputRegion(tensors)
print(len(inputRegion.variables))

input = np.random.random_sample([10,2])
print(inputRegion(input).tensor.shape)

print("\n######################################")
print("FOR MPS Test")
bond_dim = 3
feature = 2
rank = 28**2
label_size = 10
mps_Model = MPSModle(bond_dim, feature, rank, label_size)
print(len(mps_Model.variables))

######################################
FOR inputModule Test
[<tf.Variable 'Variable:0' shape=(3, 2, 3) dtype=float64, numpy=
array([[[0.80071963, 0.98507522, 0.07062338],
        [0.52477393, 0.92203361, 0.88169516]],

       [[0.46219265, 0.74761797, 0.34371726],
        [0.81399211, 0.56321762, 0.57532171]],

       [[0.64623823, 0.52813885, 0.08555296],
        [0.30156321, 0.25400102, 0.39694843]]])>]
(3, 3)

######################################
FOR outputModule Test
10

######################################
FOR inputRegion Test
10
(3, 3)

######################################
FOR MPS Test
785
